In [ ]:
import json
import os
import sys
import numpy as np
from gensim.models import Word2Vec
# from keras.preprocessing import sequence
# from keras.layers import merge, Dense, Input,Dropout, Embedding, LSTM, Bidirectional, Activation
# from keras.layers import Conv2D,Conv1D
# from keras.layers.merge import dot, multiply, add, concatenate
# from keras.layers import Merge
# from keras.layers.core import Lambda,Reshape, Flatten, Dropout
# from keras.layers.pooling import GlobalMaxPooling2D, GlobalMaxPooling1D,MaxPooling1D
# from keras.models import Model
# from keras.backend import transpose,batch_dot,expand_dims
# from keras import optimizers
from HomeDepotCSVReader import HomeDepotReader
import Utilities
from DataPreprocessing import DataPreprocessing
from Feature_Word2Vec import Feature_Word2Vec
from AutomaticQueryExpansion import Word2VecQueryExpansion
import re
from nltk.corpus import stopwords
# from keras.utils.np_utils import to_categorical
import pandas as pd
from FeatureEngineering import HomeDepotFeature
# from keras.layers.wrappers import TimeDistributed
# from keras.callbacks import ModelCheckpoint,EarlyStopping

In [ ]:
# train_filename = '../data/train_play.csv'
# test_filename = '../data/test_play.csv'
# soln_filename = '../data/solution.csv'
# attribute_filename = '../data/attributes_play.csv'
# description_filename = '../data/product_descriptions_play.csv'
# word2vec_model_path='model/word2vec_play.model'
# doc2vec_model_path='model/doc2vec_play.model'
# vocab_path='model/word2vec_play_vocab.json'
# embeddings_path='model/embeddings_play.npz'

train_filename = '../data/train.csv'
test_filename = '../data/test.csv'
soln_filename = '../data/solution.csv'
attribute_filename = '../data/attributes.csv'
description_filename = '../data/product_descriptions.csv'
word2vec_model_path='model/word2vec_sense2vec_all.model'
doc2vec_model_path='model/doc2vec_sense2vec_noun.model'
vocab_path='model/doc2vec_sense2vec_vocab_all.json'
embeddings_path='model/embeddings_sense2vec_all.npz'
full_features_filename = '../data/features_full_plusnouns_pluspuidthresh.csv'

In [ ]:
reader = HomeDepotReader()

train_query_df, product_df, attribute_df, test_query_df = reader.getQueryProductAttributeDataFrame(train_filename,
                                              test_filename,
                                              attribute_filename,
                                              description_filename)
print("train_query_df:",list(train_query_df))
print("product_df:", list(product_df))
print("attribute_df:", list(attribute_df))
print("test_query_df:", list(test_query_df))

#transform attribute into doc
dp = DataPreprocessing()
attribute_doc_df = dp.getAttributeDoc(attribute_df)
#attribute_doc_df
product_df=product_df.join(attribute_doc_df.set_index('product_uid'), on = 'product_uid')
### test data
soln_df = pd.read_csv(soln_filename, delimiter=',', low_memory=False, encoding="ISO-8859-1")
test_private_df = dp.getGoldTestSet(test_query_df, soln_df, testsetoption='Private')#,savepath='../data/test_private_gold.csv')
test_public_df = dp.getGoldTestSet(test_query_df, soln_df, testsetoption='Public')# savepath='../data/test_public_gold.csv')

In [ ]:
product_df

In [ ]:
train_query_df,product_df, attribute_df = HomeDepotFeature().getFeature(train_query_df, product_df, attribute_df, test_private_df,
                        features="spelling,nonascii,brand")

In [ ]:
test_query_df,_, _ = HomeDepotFeature().getFeature(test_query_df, product_df, attribute_df, test_private_df,
                        features="spelling,nonascii")

In [ ]:
product_df

In [ ]:
# ## no attribute
product_df['content'] = product_df['product_title'].map(str) + ". " + \
                        product_df['product_description'].map(str) 
    
    


In [ ]:
product_df['content_brand'] = product_df['product_brand']+ ". " + \
                        product_df['product_title'].map(str) + ". " + \
                        product_df['product_description'].map(str)

In [ ]:
# attribute
product_df['content_attr'] = product_df['attr_json'].map(str)
# product_df['product_brand']+ ". " + \
#                        product_df['product_title'].map(str) + ". " + \
#                         product_df['product_description'].map(str)  + ". " + \
                        

# resample

In [ ]:
# #resample remove the low classes
# resampled_train_query_df = train_query_df[train_query_df.relevance_int == 0]
# min_samples=len(resampled_train_query_df)
# for i in range(1,13):
#     print(i)
#     tmp_df = train_query_df[train_query_df.relevance_int == i]
#     if len(tmp_df) > min_samples:
#         tmp_df = tmp_df.ix[np.random.choice(tmp_df.index,min_samples)]
#         resampled_train_query_df=pd.concat([resampled_train_query_df,tmp_df])

#     else:
#         print("removing {} as too few values {}".format(i,len(tmp_df)))
        
# ori_train_query_df=train_query_df        
# train_query_df=resampled_train_query_df

In [ ]:
#don't sample
#train_query_df=ori_train_query_df


In [ ]:
#### merge test and train queries for training
orig_train_query_df = train_query_df
train_query_df = pd.concat([train_query_df,test_query_df])


In [ ]:
len(train_query_df)

# end resample

In [ ]:
import spacy
import re
#import sense2vec

In [ ]:
LABELS = {
    'ENT': 'ENT',
    'PERSON': 'ENT',
    'NORP': 'ENT',
    'FAC': 'ENT',
    'ORG': 'ENT',
    'GPE': 'ENT',
    'LOC': 'ENT',
    'LAW': 'ENT',
    'PRODUCT': 'ENT',
    'EVENT': 'ENT',
    'WORK_OF_ART': 'ENT',
    'LANGUAGE': 'ENT',
    'DATE': 'DATE',
    'TIME': 'TIME',
    'PERCENT': 'PERCENT',
    'MONEY': 'MONEY',
    'QUANTITY': 'QUANTITY',
    'ORDINAL': 'ORDINAL',
    'CARDINAL': 'CARDINAL',
}


In [ ]:
#pre_format_re = re.compile(r'^[\`\*\~]')
#post_format_re = re.compile(r'[\`\*\~]$')
#url_re = re.compile(r'\[([^]]+)\]\(%%URL\)')
#link_re = re.compile(r'\[([^]]+)\]\(https?://[^\)]+\)')
comma_num_re = re.compile(r'([0-9])(?:,)([0-9])') #Removed commas between digits (for example, 10, 000 was replaced with 10000)
digit_letter_re = re.compile(r'([0-9])([A-Za-z])') #Added dot and space between a digit (left) and a letter (right).
letters_digit_re = re.compile(r'([A-Za-z]){3}([0-9])') #Added space between at least three letters (left) and a digit (right).
letters_slash_re = re.compile(r'([A-Za-z])(?:[\\\/])([A-Za-z])') #Replaced \ or / between letters with a space.
camel_case_re = re.compile(r'([a-z])([A-Z])')
def strip_meta(text,titletext=''):
    #text = link_re.sub(r'\1', text)
    #Concatenation of Numbers with Measure Units
    text = re.sub('(?<=[0-9])[\ ]*pound[s]*(?=\ |$|\.)', '-lb ', text)
    text = re.sub('(?<=[0-9])[\ ]*lb[s]*(?=\ |$|\.)', '-lb ', text)
    text = re.sub('(?<=[0-9])[\ ]*gallon[s]*(?=\ |$|\.)', '-gal ', text)
    text = re.sub('(?<=[0-9])[\ ]*gal(?=\ |$|\.)', '-gal ', text)
    
    text = text.replace('&gt;', '>').replace('&lt;', '<').replace('&nbsp;',' ').\
            replace('&amp;','&').replace(';','.').replace(':','.').\
            replace('+',' ').replace('*','x').replace('(','.').replace(')','.')
    text = comma_num_re.sub(r'\1\2',text)    
    text = digit_letter_re.sub(r'\1. \2',text)    
    text = letters_digit_re.sub(r'\1 \2',text)  
    text = letters_slash_re.sub(r'\1 \2',text)
    if titletext:
        for word in text.split(): 
            #print(word)
            if camel_case_re.findall(word) and word not in titletext.split():  
                word_replace = camel_case_re.sub(r'\1 \2',word)
                text=re.compile(word).sub(word_replace,text)
                #print('here')
    #text = pre_format_re.sub('', text)
    #text = post_format_re.sub('', text)
    return text

In [ ]:
strip_meta('asd/sd afj\sjk asasd1100b000 hotDog 10 pounds InSinkErator','InSinkErator asd ads asd')

In [ ]:
train_query_df['search_term'] = train_query_df['search_term'].map(lambda x: strip_meta(x))
train_query_df['search_term']

In [ ]:
product_df['content_brand']=product_df.apply(lambda x: strip_meta(text=x['content_brand'],titletext=x['product_title']),axis=1)
product_df['content_brand']

In [ ]:
product_df['content_attr']=product_df.apply(lambda x: strip_meta(text=x['content_attr'],titletext=x['product_title']),axis=1)
product_df['content_attr']

# transform

In [ ]:
import spacy
#nlp = spacy.load('en')
#import en_vectors_glove_md
#nlp = en_vectors_glove_md.load()

nlp = spacy.load('en_core_web_md')

In [ ]:
sentence='BEHR_Premium_Textured_deck|NOUN over|ADP 1-gal|VERB SC|NOUN 141|CARDINAL Tugboat_Wood|ENT'
keep_tags=['NOUN','PROPN','ENT','NUM','DATE','TIME','PERCENT','MONEY','QUANTITY','ORDINAL','CARDINAL']
#a=re.compile('^(\|'+'|'.join(keep_tags)+')')
keep_tags_re=re.compile('((?:[A-Za-z0-9_]+?\|)(?:NOUN|PROPN|ENT|NUM|DATE|TIME|PERCENT|MONEY|QUANTITY|ORDINAL|CARDINAL))')
print(sentence)
print(' '.join(keep_tags_re.findall(sentence)))

In [ ]:
def represent_word(word):
    if word.like_url:
        return '%%URL|X'
    text = re.sub(r'\s', '_', word.text)
    tag = LABELS.get(word.ent_type_, word.pos_)
    if not tag:
        tag = '?'
    return text + '|' + tag

keep_tags_re=re.compile('((?:[A-Za-z0-9_]+?\|)(?:NOUN|PROPN|ENT|NUM|DATE|TIME|PERCENT|MONEY|QUANTITY|ORDINAL|CARDINAL))')
def transform_doc(doc,filter_pos=False):
    for ent in doc.ents:
        ent.merge(ent.root.tag_, ent.text, LABELS[ent.label_])
    for np in doc.noun_chunks:
        while len(np) > 1 and np[0].dep_ not in ('advmod', 'amod', 'compound'):
            np = np[1:]
        np.merge(np.root.tag_, np.text, np.root.ent_type_)
    strings = []
    for sent in doc.sents:
        if sent.text.strip():         
            if filter_pos == True:
                s = ' '.join(represent_word(w) for w in sent if not w.is_space)
                strings.append(' '.join(keep_tags_re.findall(s)))
            else:
                strings.append(' '.join(represent_word(w) for w in sent if not w.is_space))
    if strings:
        return '\n'.join(strings) + '\n'
    else:
        return ''

In [ ]:
import time
from Utilities import Utility
#timetracker=Utility()

i=0
for row in product_df.itertuples():
    #timetracker.startTimeTrack()
    #start_time = time.time()
    print(row.content_brand)
    print("=========")
    doc=nlp(row.content_brand)
    print(transform_doc(doc,filter_pos=False))
    i+=1
    #timetracker.checkpointTimeTrack()
    #print("%s s" % round(((time.time() - start_time) ), 2))
    if i > 1:
        break

# product_df['sense2vec_content_attr']=product_df.apply(lambda x: transform_doc(nlp(x['content_attr'])),axis=1)
# product_df['sense2vec_content_attr']

In [ ]:
from joblib import Parallel, delayed
import multiprocessing
    
# what are your inputs, and what operation do you want to 
# perform on each input. For example...
def processInput(i):
    return transform_doc(nlp(i),filter_pos=False)

num_cores = multiprocessing.cpu_count()
    
results = Parallel(n_jobs=num_cores)(delayed(processInput)(i.content_brand) for i in product_df.itertuples())

In [ ]:
sense2vec_content_brand_df=pd.DataFrame(results,columns=['sense2vec_content_brand'])
product_df=pd.concat([product_df,sense2vec_content_brand_df],axis=1)

In [ ]:
product_df['sense2vec_content_brand'][0]

In [ ]:
# product_df['sense2vec_content_brand']=product_df.apply(lambda x: transform_doc(nlp(x['content_brand'])),axis=1)
# product_df['sense2vec_content_brand']

In [ ]:
train_query_df['sense2vec_search_term']=train_query_df.apply(lambda x: transform_doc(nlp(x['search_term']),filter_pos=False),axis=1)
train_query_df['sense2vec_search_term']

# results = Parallel(n_jobs=num_cores)(delayed(processInput)(i.search_term) for i in train_query_df.itertuples())
# sense2vec_search_term_df=pd.DataFrame(results,columns=['sense2vec_search_term'])
# train_query_df=pd.concat([train_query_df,sense2vec_search_term_df],axis=1)

In [ ]:
train_query_df

# prep for word2vec

In [ ]:
sentences = []  # Initialize an empty list of sentences
query_sentences = []
doc_sentences = []

print("Parsing sentences from search string")
for query in train_query_df["sense2vec_search_term"]:
    query_sentences += [query.split()]

print("Parsing sentences from unlabeled set")
for doc in product_df['sense2vec_content_brand']:
    doc_sentences += [doc.split()]

sentences = query_sentences+doc_sentences 

In [ ]:
sentences[0:2]

In [ ]:
sentences[-3:-1]

# word2vec

In [ ]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300#50#300    # Word vector dimensionality                      
min_word_count = 1#5#40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-5 #0 #1e-3   # Downsample setting for frequent words
iterations = 10

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print("Training model...")
word2vec_model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling, iter=iterations)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
word2vec_model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
#model_name = "300features_40minwords_10context"
word2vec_model.save(word2vec_model_path)

In [ ]:
print(len(word2vec_model.wv.vocab))

In [ ]:
vocab_words=[k for k,v in word2vec_model.wv.vocab.items()]

In [ ]:
print(word2vec_model.most_similar('wood|NOUN', [], 5))

In [ ]:
print(word2vec_model.most_similar('Whirlpool|ENT', [], 3))

In [ ]:
print(word2vec_model.most_similar('kitchen|NOUN', [], 3))

In [ ]:
print(word2vec_model.most_similar('shower|NOUN', [], 3))

In [ ]:
print(word2vec_model.most_similar('steel|NOUN', [], 10))

In [ ]:
print(word2vec_model.most_similar('microwave|NOUN', [], 10))

In [ ]:
print(word2vec_model.most_similar('LED|NOUN', [], 10))

In [ ]:
print(word2vec_model.most_similar('stool|NOUN', [], 10))

# word2vec cosine similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

i=0
#all_similarity_scores=[]
#keep_tag_similarity_scores=[]
avg_all_similarity_scores=[]
avg_keep_tag_similarity_scores=[]
avg_uidfact_all_similarity_scores=[]
avg_uidfact_keep_tag_similarity_scores=[]
relevance_scores=[]
for query_row in train_query_df.itertuples():    
    query=nlp(str(query_row.search_term)) 
    relevance=query_row.relevance
    product_uid=int(query_row.product_uid)
    #product_idx=query_row.product_idx.values[0]
    product_idx=product_df[product_df['product_uid'] == product_uid]['content_brand'].index[0] #feature_df workaround
    
    if product_df.iloc[product_idx]['product_uid'] < 163100:
        product_uid_factor=1.0
    else:
        product_uid_factor=0.75
        
    ## similarity doc
    #similarity_score = doc2vec_model.docvecs.similarity(i,int(len(train_query_df)+product_idx))
    
    ### cosine
    keep_tag_similarity_score=0
    all_similarity_score=0
    uidfact_keep_tag_similarity_score=0
    uidfact_all_similarity_score=0
    for word in query_row.sense2vec_search_term.split():        
        #print(word2vec_model.most_similar(word, [], 5))
        for doc_word in product_df.iloc[product_idx]['sense2vec_content_brand'].split():
            similarity = word2vec_model.similarity(word,doc_word)
            #print("{:.2f} = {} {} to {} ".format(similarity,relevance,word,doc_word))
            if keep_tags_re.findall(doc_word):
                keep_tag_similarity_score+=similarity
                uidfact_keep_tag_similarity_score+=similarity*product_uid_factor
                #print('keep_tag score = {}'.format(keep_tag_similarity_score))
            all_similarity_score+=similarity
            uidfact_all_similarity_score+=similarity*product_uid_factor
        
        #print(doc2vec_model[word].reshape(-1,1).T.shape)
        #print(doc2vec_model.docvecs[int(len(train_query_df)+product_idx)].reshape(-1,1).T.shape)
#         similarity_score += np.squeeze(cosine_similarity(doc2vec_model[word].reshape(-1,1).T,\
#                           doc2vec_model.docvecs[int(len(train_query_df)+product_idx)].reshape(-1,1).T))
#         print(np.squeeze(cosine_similarity(doc2vec_model[word].reshape(-1,1).T,\
#                          doc2vec_model.docvecs[int(len(train_query_df)+product_idx)].reshape(-1,1).T)))
#       print(similarity_score)

    ### end cosine
    avg_all_similarity_scores+=[all_similarity_score/(len(query_row.sense2vec_search_term.split())*len(product_df.iloc[product_idx]['sense2vec_content_brand'].split()))]
    avg_keep_tag_similarity_scores+=[keep_tag_similarity_score/(len(query_row.sense2vec_search_term.split())*len(product_df.iloc[product_idx]['sense2vec_content_brand'].split()))]
    avg_uidfact_all_similarity_scores+=[uidfact_all_similarity_score/(len(query_row.sense2vec_search_term.split())*len(product_df.iloc[product_idx]['sense2vec_content_brand'].split()))]
    avg_uidfact_keep_tag_similarity_scores+=[uidfact_keep_tag_similarity_score/(len(query_row.sense2vec_search_term.split())*len(product_df.iloc[product_idx]['sense2vec_content_brand'].split()))]
    #all_similarity_scores+=[all_similarity_score]
    #keep_tag_similarity_scores+=[keep_tag_similarity_score]
    relevance_scores+=[relevance]
    
    i+=1
    if i % 1000 == 0: #print every 100 
        print(i)        
#     if i >= 200:
#         print(relevance,all_similarity_score,keep_tag_similarity_score)
#         break
#     else:
#         print(relevance,all_similarity_score,keep_tag_similarity_score)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
fig = plt.gcf()
fig.set_size_inches(8,15)
#sns.stripplot(relevance_scores,similarity_scores,jitter=True, alpha=.40)
#sns.pointplot(relevances,overlap_ratios,jitter=True, alpha=.40)
sns.boxplot(relevance_scores,avg_uidfact_all_similarity_scores)

#sns.despine()

In [ ]:
fig = plt.gcf()
fig.set_size_inches(8,15)
#sns.stripplot(relevance_scores,similarity_scores,jitter=True, alpha=.40)
#sns.pointplot(relevances,overlap_ratios,jitter=True, alpha=.40)
sns.boxplot(relevance_scores,avg_uidfact_keep_tag_similarity_scores)

#sns.despine()

In [ ]:
fig = plt.gcf()
fig.set_size_inches(8,15)
#sns.stripplot(relevance_scores,similarity_scores,jitter=True, alpha=.40)
#sns.pointplot(relevances,overlap_ratios,jitter=True, alpha=.40)
sns.boxplot(relevance_scores,avg_all_similarity_scores)

#sns.despine()

In [ ]:
fig = plt.gcf()
fig.set_size_inches(8,15)
#sns.stripplot(relevance_scores,similarity_scores,jitter=True, alpha=.40)
#sns.pointplot(relevances,overlap_ratios,jitter=True, alpha=.40)
sns.boxplot(relevance_scores,avg_keep_tag_similarity_scores)

#sns.despine()

# save results

In [ ]:
id_df = pd.DataFrame(train_query_df['id'],index=train_query_df.index)

In [ ]:
ref_index=train_query_df.index
new_feature_df1 = pd.DataFrame(avg_all_similarity_scores,columns=['sense2vec_all_simscore'],index=ref_index)
new_feature_df2 = pd.DataFrame(avg_keep_tag_similarity_scores,columns=['sense2vec_keeptag_simscore'],index=ref_index)
new_feature_df3 = pd.DataFrame(avg_uidfact_all_similarity_scores,columns=['sense2vec_uidfact_all_simscore'],index=ref_index)
new_feature_df4 = pd.DataFrame(avg_uidfact_keep_tag_similarity_scores,columns=['sense2vec_uidfact_keeptag_simscore'],index=ref_index)
new_feature_df = pd.concat([id_df,new_feature_df1,new_feature_df2,new_feature_df3,new_feature_df4],axis=1)


In [ ]:
new_feature_df

In [ ]:
pd.DataFrame(new_feature_df).to_csv('../data/features_sense2vec_simscore_attr.csv', \
                                    index=False, header=True)

In [ ]:
# save_all_similarity_scores = all_similarity_scores
# save_keep_tag_similarity_scores = keep_tag_similarity_scores
# save_avg_all_similarity_scores=avg_all_similarity_scores
# save_avg_keep_tag_similarity_scores=avg_keep_tag_similarity_scores

# concat to full features

In [ ]:
feature_df = reader.getBasicDataFrame(full_features_filename)

In [ ]:
feature_df

In [ ]:
copy_new_feature_df = new_feature_df.copy()
_ = copy_new_feature_df.pop('id')
copy_new_feature_df.reset_index(drop=True,inplace=True)

In [ ]:
copy_new_feature_df

In [ ]:
feature_df2 = pd.concat([feature_df,copy_new_feature_df],axis=1)
#copy_new_feature_df
#feature_df2 = feature_df.join(new_feature_df,on='id',how='inner',rsuffix='rid_')

#feature_df2=feature_df.set_index('id').join(feature_df2.set_index('id'),lsuffix='_')

In [ ]:
#_=feature_df2.pop('idrid_')

In [ ]:
feature_df2

In [ ]:
pd.DataFrame(feature_df2).to_csv('../data/features_full_plusnouns_pluspuidthreshpluss2vsimscorev2.csv', \
                                    index=False, header=True)

# concat to latests features

In [ ]:
latest_feature_df = reader.getBasicDataFrame('../data/features_doc2vec_sense2vec_20170416.csv')

In [ ]:
latest_feature_df

In [ ]:
new_feature_df

In [ ]:
feature_df[['id','product_uid_threshold']]

In [ ]:
feature_df3 = pd.concat([latest_feature_df,copy_new_feature_df],axis=1)

# feature_df3 = latest_feature_df.join(new_feature_df,on='id',how='inner',rsuffix='rid_')
# _=feature_df3.pop('idrid_')
# feature_df3

In [ ]:
feature_df3 = pd.concat([feature_df3,feature_df['product_uid_threshold']],axis=1)

# feature_df3 = feature_df3.join(feature_df[['id','product_uid_threshold']],on='id',how='inner',rsuffix='rid_')
# _=feature_df3.pop('idrid_')
# feature_df3

In [ ]:
feature_df3

In [ ]:
pd.DataFrame(feature_df3).to_csv('../data/features_doc2vec_sense2vec_20170417.csv', \
                                    index=False, header=True)

# prep for doc2vec

In [ ]:
sentences = []  # Initialize an empty list of sentences
query_sentences = []
doc_sentences = []

print("Parsing sentences from search string")
for query in train_query_df["sense2vec_search_term"]:
    query_sentences += [query.split()]

print("Parsing sentences from unlabeled set")
for doc in product_df['sense2vec_content_brand']:
    doc_sentences += [doc.split()]

sentences = query_sentences+doc_sentences    

In [ ]:
sentences[0:2]

In [ ]:
sentences[-3:-1]

In [ ]:
print(len(sentences))

from gensim.models import doc2vec #word2vec

documents=[]
for i,s in enumerate(sentences):
    if s:
        document={}
        document= doc2vec.LabeledSentence(words=s,tags=['SENT_'+str(i)]) 
        documents+=[document]
print(documents[-1])    

In [ ]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 100#50#300    # Word vector dimensionality                      
min_word_count = 1#5#40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 10e-5 #0 #1e-3   # Downsample setting for frequent words

#https://github.com/jhlau/doc2vec hyperparam guide

# Initialize and train the model (this will take some time)
from gensim.models import doc2vec #word2vec
print("Training model...")
# doc2vec_model = doc2vec.Doc2Vec(documents, workers=num_workers, \
#             size=num_features, min_count = min_word_count, \
#             window = context, sample = downsampling,iter=20)

# doc2vec_model = doc2vec.Doc2Vec(documents, size=300, min_count=5, window=5, \
#                             iter=30, workers=4, alpha=0.1, min_alpha=0.0001,
#                         dm_concat=1, dm=0, negative=5)

# 20:43 run
# doc2vec_model = doc2vec.Doc2Vec(documents, size=100, min_count=0, window=8, \
#                             iter=20, workers=4, #alpha=0.1, min_alpha=0.0001,
#                             dm_concat=1, dm=0,sample = 0,  negative=5,dbow_words=1)  

# # 20:57 run
# doc2vec_model = doc2vec.Doc2Vec(documents, size=100, min_count=0, window=16, \
#                             iter=20, workers=4, #alpha=0.1, min_alpha=0.0001,
#                             dm_concat=1, dm=0,sample = downsampling,  negative=5,dbow_words=1)  

#New
doc2vec_model = doc2vec.Doc2Vec(documents, size=200, min_count=1, window=10, \
                            iter=30, workers=4, #alpha=0.1, min_alpha=0.0001,
                            sample = downsampling,  negative=5,dbow_words=1,
                               dm_mean=0, dm_concat=1, dm=1,)  

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
doc2vec_model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
#model_name = "300features_40minwords_10context"
doc2vec_model.save(doc2vec_model_path)

In [ ]:
print(len(doc2vec_model.wv.vocab))
doc2vec_model.wv.vocab

In [ ]:
vocab_words=[k for k,v in doc2vec_model.wv.vocab.items()]

In [ ]:
train_query_df.iloc[0]

In [ ]:
r = re.compile("r_emote\|",flags=re.IGNORECASE)
newlist = filter(r.match, vocab_words)
print(list(newlist))

In [ ]:
print(doc2vec_model.most_similar('r_emote|NOUN', [], 3))

In [ ]:
print(doc2vec_model.docvecs.most_similar( [ doc2vec_model['r_emote|NOUN'] ] ))

In [ ]:
print(sentences[0])
print(sentences[len(train_query_df)])

In [ ]:
doc2vec_model.docvecs.most_similar(['SENT_0'])

In [ ]:
print(sentences[74067])

In [ ]:
#doc2vec_model.docvecs.similarity('SENT_17','SENT_74068')
doc2vec_model.docvecs.similarity(0,74067+11)

In [ ]:

len(train_query_df)

In [ ]:
train_query_df

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

i=0
similarity_scores=[]
relevance_scores=[]
for query_row in train_query_df.itertuples():    
    query=nlp(str(query_row.search_term)) 
    relevance=query_row.relevance
    product_uid=int(query_row.product_uid)
    #product_idx=query_row.product_idx.values[0]
    product_idx=product_df[product_df['product_uid'] == product_uid]['content_attr'].index[0] #feature_df workaround
    
    ## similarity doc
    similarity_score = doc2vec_model.docvecs.similarity(i,int(len(train_query_df)+product_idx))
    
    ### cosine
#     similarity_score=0
#     for word in query_row.sense2vec_search_term.split():
#         #print(word)
#         #print(doc2vec_model[word].reshape(-1,1).T.shape)
#         #print(doc2vec_model.docvecs[int(len(train_query_df)+product_idx)].reshape(-1,1).T.shape)
#         similarity_score += np.squeeze(cosine_similarity(doc2vec_model[word].reshape(-1,1).T,\
#                           doc2vec_model.docvecs[int(len(train_query_df)+product_idx)].reshape(-1,1).T))
# #         print(np.squeeze(cosine_similarity(doc2vec_model[word].reshape(-1,1).T,\
# #                          doc2vec_model.docvecs[int(len(train_query_df)+product_idx)].reshape(-1,1).T)))
# #       print(similarity_score)

    ### end cosine
    
    similarity_scores+=[similarity_score]
    relevance_scores+=[relevance]
    
    i+=1
    if i % 1000 == 0: #print every 100 
        print(i)
#     if i >= 10:
#         break
#     else:
#         print(relevance,similarity_score)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
fig = plt.gcf()
fig.set_size_inches(8,15)
#sns.stripplot(relevance_scores,similarity_scores,jitter=True, alpha=.40)
#sns.pointplot(relevances,overlap_ratios,jitter=True, alpha=.40)
sns.boxplot(relevance_scores,similarity_scores)

#sns.despine()

In [ ]:
# from gensim.models.word2vec import Word2Vec
# from gensim.models.word2vec import KeyedVectors
# word2vec_model = Word2Vec.load('/home/ongmin/PycharmProjects/IRDM2017/data/models/prod_content_model')


In [ ]:
#word2vec_model.wv.vocab

In [ ]:
print(doc2vec_model.most_similar('Whirlpool|ENT', [], 3))

In [ ]:
print(doc2vec_model.most_similar('wood|NOUN', [], 3))

In [ ]:
print(doc2vec_model.most_similar('kitchen|NOUN', [], 3))

In [ ]:
print(doc2vec_model.most_similar('shower|NOUN', [], 3))

In [ ]:
print(doc2vec_model.most_similar('steel|NOUN', [], 10))

In [ ]:
print(doc2vec_model.most_similar('microwave|NOUN', [], 10))

In [ ]:
print(doc2vec_model.most_similar('LED|NOUN', [], 10))

In [ ]:
words=[k for k,v in word2vec_model.wv.vocab.items()]

In [ ]:
r = re.compile("stool\|",flags=re.IGNORECASE)
newlist = filter(r.match, words)
print(list(newlist))

In [ ]:
for word in list(filter(r.match, words)):
    #print(str(word))
    print(word,word2vec_model.most_similar(word, [], 3))

In [ ]:
nlp = spacy.load('en')
nlp.vocab.load_vectors("/home/ongmin/PycharmProjects/IRDM2017/data/models/prod_content_model")

In [ ]:
from sense2vec.vectors import VectorMap
vector_map = VectorMap(300)
a=vector_map.load('/home/ongmin/PycharmProjects/IRDM2017/data/models/')

In [ ]:
model = sense2vec.load('/home/ongmin/PycharmProjects/IRDM2017/data/models/prod_content_model')